In [ ]:
import pandas as pd

# Läs in CSV-filen
exercise_data = pd.read_csv('cleaned_exercise_file.csv')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

# Välj relevanta kolumner för klustring
features_to_cluster = exercise_data[['Main_muscle', 'Target_Muscles']]

# Använd OneHotEncoder för att omvandla dessa kolumner till numeriska värden
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(features_to_cluster)

# Använd K-means för att klustra övningarna
kmeans = KMeans(n_clusters=10, random_state=42)  # Välj antal kluster till 11
exercise_data['Cluster'] = kmeans.fit_predict(encoded_features)

# Visa övningar med deras kluster
# print(exercise_data[['Exercise Name', 'Cluster']])
# Visa övningar grupperade efter kluster
clustered_exercises = exercise_data.groupby('Cluster')['Exercise Name'].apply(list)
print(clustered_exercises)

output_file = 'exercise_clusters.xlsx'
exercise_data.to_excel(output_file, index=False)

In [29]:
import pandas as pd

# Anta att exercise_data redan har laddats med övningar och deras information

# Exempel på träningsschema
training_schedule = {
    "MONDAY": ["Front Raise", "Triceps Extension", "Military Press"],  # Måndag med övningar
    "TUESDAY": ["Military Press", "Hammer Curl", "Grip", "One Arm Straight Back Seated High Row", "Bent-over Row"],
    "WEDNESDAY": ["Front Raise", "Triceps Extension"],  # Exempel på duplicerade övningar
    "THURSDAY": ["Front Raise", "Triceps Extension", "Military Press"],
    "FRIDAY": [],
    "SATURDAY": [],
    "SUNDAY": [],
}

# Skapa en ny lista för att lagra det slutliga schemat
final_schedule = []

# Lista över veckodagar för att underlätta indexering
days_of_week = list(training_schedule.keys())

# Uppsättning för att hålla reda på använda övningar från föregående dag
previous_used_exercises = set()

# Iterera över schemat
for i, day in enumerate(days_of_week):
    exercises = training_schedule[day]
    used_exercises = set()  # Skapa en ny uppsättning för den aktuella dagen

    if exercises:  # Om det finns övningar för dagen
        for exercise in exercises:
            # Hämta information om övningen
            exercise_info = exercise_data.loc[exercise_data['Exercise Name'] == exercise]
            if not exercise_info.empty:
                exercise_name = exercise_info['Exercise Name'].values[0]  # Hämta övningsnamnet
                description = exercise_info['Execution'].values[0] if 'Execution' in exercise_info else ""
                equipment = exercise_info['Equipment'].values[0] if 'Equipment' in exercise_info else ""

                # Kontrollera om övningen redan har använts på föregående dag
                if i > 0 and exercise_name in previous_used_exercises:
                    # Filtrera för att hitta en annan övning i samma kluster
                    cluster_number = exercise_info['Cluster'].values[0]  # Hämta klustret
                    alternative_exercises = exercise_data[exercise_data['Cluster'] == cluster_number]
                    alternative_exercises = alternative_exercises[~alternative_exercises['Exercise Name'].isin(used_exercises | previous_used_exercises)]

                    if not alternative_exercises.empty:
                        # Välj en alternativ övning
                        alternative_exercise = alternative_exercises.sample(n=1)  # Välj slumpmässigt en övning
                        exercise_name = alternative_exercise['Exercise Name'].values[0]
                        description = alternative_exercise['Execution'].values[0] if 'Execution' in alternative_exercise else ""
                        equipment = alternative_exercise['Equipment'].values[0] if 'Equipment' in alternative_exercise else ""

                # Lägg till övningen i final_schedule
                final_schedule.append({
                    "Day": day,
                    "Exercise Name": exercise_name,
                    "Description": description,
                    "Equipment": equipment
                })

                # Lägg till övningen i used_exercises för den aktuella dagen
                used_exercises.add(exercise_name)

        # Efter att ha bearbetat övningarna för dagen, uppdatera previous_used_exercises
        previous_used_exercises = used_exercises.copy()

# Konvertera till DataFrame för att visa
final_schedule_df = pd.DataFrame(final_schedule)
print(final_schedule_df)

output_file = 'final_scheme.xlsx'
final_schedule_df.to_excel(output_file, index=False)

          Day                               Exercise Name  \
0      MONDAY                                 Front Raise   
1      MONDAY                           Triceps Extension   
2      MONDAY                              Military Press   
3     TUESDAY                                 Upright Row   
4     TUESDAY                                 Hammer Curl   
5     TUESDAY                                        Grip   
6     TUESDAY       One Arm Straight Back Seated High Row   
7     TUESDAY                               Bent-over Row   
8   WEDNESDAY                                 Front Raise   
9   WEDNESDAY                           Triceps Extension   
10   THURSDAY  Seated Rear Delt Row:  alternative machine   
11   THURSDAY                      Close Grip Bench Press   
12   THURSDAY                              Military Press   

                                          Description             Equipment  
0   Raise barbell forward and upward until upper a...              